In [1]:

%matplotlib inline
import matplotlib.pyplot as plt
from fastai.vision import *
from fastai.metrics import accuracy
from fastai.basic_data import *
from skimage.util import montage
import pandas as pd
from torch import optim
import re

from utils import *

In [2]:
df = pd.read_csv('data/train.csv')
df.head()

,Image,Id
0,0000e88ab.jpg,w_f48451c
1,0001f9222.jpg,w_c3d896a
2,00029d126.jpg,w_20df2c5
3,00050a15a.jpg,new_whale
4,0005c1ef8.jpg,new_whale


In [3]:
im_count = df[df.Id != 'new_whale'].Id.value_counts()
im_count.name = 'sighting_count'
df = df.join(im_count, on='Id')
#大于1个样本的图像
val_fns = set(df.sample(frac=1)[(df.Id != 'new_whale') & (df.sighting_count > 1)].groupby('Id').first().Image)

/home/astatus/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [12]:
set(df.sample(frac=1)

SyntaxError: unexpected EOF while parsing (<ipython-input-12-919bd5f5806b>, line 1)

In [13]:
n2label = {row[1].Image: row[1].Id for row in df.iterrows()}

In [14]:
SZ = 224
BS = 64
NUM_WORKERS = 12
SEED=0

In [15]:
path2fn = lambda path: re.search('\w*\.jpg$', path).group(0)

In [16]:
path2fn

<function __main__.<lambda>(path)>

In [17]:
df = df[df.Id != 'new_whale']

In [19]:
df.shape

(15697, 3)

In [20]:
df.sighting_count.max()

73.0

In [21]:
df_val = df[df.Image.isin(val_fns)]
df_train = df[~df.Image.isin(val_fns)]
df_train_with_val = df

In [22]:
df_val.shape, df_train.shape, df_train_with_val.shape

((2931, 3), (12766, 3), (15697, 3))

In [23]:
%%time

res = None
sample_to = 15

for grp in df_train.groupby('Id'):
    n = grp[1].shape[0]
    additional_rows = grp[1].sample(0 if sample_to < n  else sample_to - n, replace=True)
    rows = pd.concat((grp[1], additional_rows))
    
    if res is None: res = rows
    else: res = pd.concat((res, rows))

CPU times: user 8.97 s, sys: 0 ns, total: 8.97 s
Wall time: 8.97 s


In [34]:
for grp in df_train.groupby('Id'):
    if grp[1].shape[0] == 2:
        print(grp[1].sample(0 if sample_to < n  else sample_to - n, replace=True))

               Image         Id  sighting_count
18174  b6918b6a7.jpg  w_01cb0c4             3.0
10057  65d9f620f.jpg  w_01cb0c4             3.0
10057  65d9f620f.jpg  w_01cb0c4             3.0
10057  65d9f620f.jpg  w_01cb0c4             3.0
18174  b6918b6a7.jpg  w_01cb0c4             3.0
10057  65d9f620f.jpg  w_01cb0c4             3.0
18174  b6918b6a7.jpg  w_01cb0c4             3.0
18174  b6918b6a7.jpg  w_01cb0c4             3.0
10057  65d9f620f.jpg  w_01cb0c4             3.0
18174  b6918b6a7.jpg  w_01cb0c4             3.0
10057  65d9f620f.jpg  w_01cb0c4             3.0
10057  65d9f620f.jpg  w_01cb0c4             3.0
18174  b6918b6a7.jpg  w_01cb0c4             3.0
18174  b6918b6a7.jpg  w_01cb0c4             3.0
               Image         Id  sighting_count
18397  b8e258cd5.jpg  w_036bc54             3.0
9621   618221292.jpg  w_036bc54             3.0
18397  b8e258cd5.jpg  w_036bc54             3.0
18397  b8e258cd5.jpg  w_036bc54             3.0
9621   618221292.jpg  w_036bc54         

               Image         Id  sighting_count
3508   2405d53b9.jpg  w_2aaa3b5             3.0
3508   2405d53b9.jpg  w_2aaa3b5             3.0
3508   2405d53b9.jpg  w_2aaa3b5             3.0
3508   2405d53b9.jpg  w_2aaa3b5             3.0
3508   2405d53b9.jpg  w_2aaa3b5             3.0
3508   2405d53b9.jpg  w_2aaa3b5             3.0
3508   2405d53b9.jpg  w_2aaa3b5             3.0
10823  6ddcdb6b0.jpg  w_2aaa3b5             3.0
3508   2405d53b9.jpg  w_2aaa3b5             3.0
10823  6ddcdb6b0.jpg  w_2aaa3b5             3.0
10823  6ddcdb6b0.jpg  w_2aaa3b5             3.0
3508   2405d53b9.jpg  w_2aaa3b5             3.0
10823  6ddcdb6b0.jpg  w_2aaa3b5             3.0
10823  6ddcdb6b0.jpg  w_2aaa3b5             3.0
               Image         Id  sighting_count
20245  cbd894388.jpg  w_2b388fe             3.0
5856   3bfde7c49.jpg  w_2b388fe             3.0
5856   3bfde7c49.jpg  w_2b388fe             3.0
5856   3bfde7c49.jpg  w_2b388fe             3.0
5856   3bfde7c49.jpg  w_2b388fe         

               Image         Id  sighting_count
6216   3f89d4e99.jpg  w_68f23a1             3.0
6216   3f89d4e99.jpg  w_68f23a1             3.0
12165  7b382c103.jpg  w_68f23a1             3.0
6216   3f89d4e99.jpg  w_68f23a1             3.0
12165  7b382c103.jpg  w_68f23a1             3.0
12165  7b382c103.jpg  w_68f23a1             3.0
12165  7b382c103.jpg  w_68f23a1             3.0
6216   3f89d4e99.jpg  w_68f23a1             3.0
12165  7b382c103.jpg  w_68f23a1             3.0
12165  7b382c103.jpg  w_68f23a1             3.0
12165  7b382c103.jpg  w_68f23a1             3.0
12165  7b382c103.jpg  w_68f23a1             3.0
12165  7b382c103.jpg  w_68f23a1             3.0
12165  7b382c103.jpg  w_68f23a1             3.0
               Image         Id  sighting_count
4504   2e171e60e.jpg  w_695c604             3.0
4504   2e171e60e.jpg  w_695c604             3.0
15550  9ccd06b4e.jpg  w_695c604             3.0
4504   2e171e60e.jpg  w_695c604             3.0
15550  9ccd06b4e.jpg  w_695c604         

              Image         Id  sighting_count
3153  206715dab.jpg  w_954fec8             3.0
3153  206715dab.jpg  w_954fec8             3.0
3153  206715dab.jpg  w_954fec8             3.0
1143  0b39157f8.jpg  w_954fec8             3.0
1143  0b39157f8.jpg  w_954fec8             3.0
1143  0b39157f8.jpg  w_954fec8             3.0
1143  0b39157f8.jpg  w_954fec8             3.0
3153  206715dab.jpg  w_954fec8             3.0
1143  0b39157f8.jpg  w_954fec8             3.0
3153  206715dab.jpg  w_954fec8             3.0
1143  0b39157f8.jpg  w_954fec8             3.0
3153  206715dab.jpg  w_954fec8             3.0
1143  0b39157f8.jpg  w_954fec8             3.0
3153  206715dab.jpg  w_954fec8             3.0
               Image         Id  sighting_count
16251  a3ce5e614.jpg  w_95b8b64             3.0
16251  a3ce5e614.jpg  w_95b8b64             3.0
16251  a3ce5e614.jpg  w_95b8b64             3.0
7978   508f7822f.jpg  w_95b8b64             3.0
7978   508f7822f.jpg  w_95b8b64             3.0
7978   

               Image         Id  sighting_count
10264  67df3fc70.jpg  w_d853d93             3.0
23081  e8a547e25.jpg  w_d853d93             3.0
10264  67df3fc70.jpg  w_d853d93             3.0
23081  e8a547e25.jpg  w_d853d93             3.0
10264  67df3fc70.jpg  w_d853d93             3.0
10264  67df3fc70.jpg  w_d853d93             3.0
23081  e8a547e25.jpg  w_d853d93             3.0
23081  e8a547e25.jpg  w_d853d93             3.0
23081  e8a547e25.jpg  w_d853d93             3.0
10264  67df3fc70.jpg  w_d853d93             3.0
10264  67df3fc70.jpg  w_d853d93             3.0
23081  e8a547e25.jpg  w_d853d93             3.0
23081  e8a547e25.jpg  w_d853d93             3.0
10264  67df3fc70.jpg  w_d853d93             3.0
              Image         Id  sighting_count
3221  2115c5048.jpg  w_d8cb231             3.0
9103  5c5e91098.jpg  w_d8cb231             3.0
9103  5c5e91098.jpg  w_d8cb231             3.0
9103  5c5e91098.jpg  w_d8cb231             3.0
9103  5c5e91098.jpg  w_d8cb231             3.

In [24]:
%%time

res_with_val = None
sample_to = 15

for grp in df_train_with_val.groupby('Id'):
    n = grp[1].shape[0]
    additional_rows = grp[1].sample(0 if sample_to < n  else sample_to - n, replace=True)
    rows = pd.concat((grp[1], additional_rows))
    
    if res_with_val is None: res_with_val = rows
    else: res_with_val = pd.concat((res_with_val, rows))

CPU times: user 8.64 s, sys: 5.1 ms, total: 8.65 s
Wall time: 8.65 s


In [25]:
res.shape, res_with_val.shape

((76174, 3), (76287, 3))